In [1]:
import pandas as pd
df=pd.read_csv("abcd - data.csv")

In [2]:
df.head()

,Email_Subject,Job_Application
0,Thank you for applying to Dealozo for internship,Yes
1,A third-party OAuth application has been added...,No
2,Auto response: AWS Educate Application Not App...,No
3,Thank you for applying to Stirring Minds,Yes
4,Earn your MCS from ASU - Early application dea...,No


In [3]:
df['label'] = df['Job_Application'].apply(lambda x: 0 if x=='No' else 1)

In [4]:
df.head()

,Email_Subject,Job_Application,label
0,Thank you for applying to Dealozo for internship,Yes,1
1,A third-party OAuth application has been added...,No,0
2,Auto response: AWS Educate Application Not App...,No,0
3,Thank you for applying to Stirring Minds,Yes,1
4,Earn your MCS from ASU - Early application dea...,No,0


In [5]:
!pip install spacy

!python -m spacy download en

     |████████████████████████████████| 9.9 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 119 kB 38.9 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 9.9 MB/s  eta 0:00:01
     |████████████████████████████████| 185 kB 48.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.7 MB 16.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 70.4 MB/s eta 0:00:01
     |████████████████████████████████| 12.0 MB 1.2 MB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047105 sha256=bc982f300e686555e91f29ac108812d98a197a84ef5af7a438c63d1e5acf12a3
  Stored in directory: /tmp/pip-ephem-wheel-cache-x_xaqe5r/wheels/10/6f/a6/ddd8204ceecdedddea923f8514e13afb0c1f0f556d2c9c3da0
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/srv/conda/envs/notebook/lib/python3.6/site-packages/en_core_web_sm --

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [7]:
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens


In [8]:
from spacy.lang.en import English

In [9]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [10]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [11]:
from sklearn.model_selection import train_test_split

X = df['Email_Subject'] # the features we want to analyze
ylabels = df['label'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

In [12]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x7f9849347160>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7f984a5b7158>)),
                ('classifier', LogisticRegression())])

In [13]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.875
Logistic Regression Precision: 1.0
Logistic Regression Recall: 0.8


In [14]:
[X_test, y_test]

[42     Thank you for applying to Chipmonk Technologies 
 25                  Thank you for applying to Rightbot 
 21                Thank You for applying to Whatscutpro
 13                   What should you not do in your 20s
 14                   Thanks for your application, Ajith
 41    Thank you for applying to Simpl and successful...
 37                          Are You Looking For PSU Job
 32                     Arun Kumar followed you on Quora
 43    Thank you for applying in the role of Computer...
 30    Thank you for applying at SigTuple for AI Inte...
 1     A third-party OAuth application has been added...
 12                    Thank you for applying at Siemens
 39                       Thanks you for applying, AJITH
 4     Earn your MCS from ASU - Early application dea...
 33                 Thank you for applying at HyperVerge
 46    3 internships mandatory to get your degree - H...
 Name: Email_Subject, dtype: object,
 42    1
 25    1
 21    1
 13    0
 14    1
 41   

In [15]:
pipe.predict(["Thank you for applying for the role of ML intern"])

array([1])